In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as pt
import os
import sys

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [3]:
from scripts.utils import dataLoader
from scripts.queries import userInfoAgregation

In [4]:
dloader = dataLoader()


In [5]:
# Define your SQL query
query = "SELECT * FROM xdr_data;"

In [6]:
# Load data from PostgreSQL using SQLAlchemy
df = dataLoader.load_data_using_sqlalchemy(query)

# Display the first few rows of the dataframe
if df is not None:
    print("Successfully loaded the data")
else:
    print("Failed to load data.")

query executed successfully!
Successfully loaded the data


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

In [8]:
query2 = """
        SELECT
            "IMSI" AS user_id,
            COUNT(*) AS num_xdr_sessions,
            SUM("Dur. (ms)") AS total_session_duration_ms,
            SUM("Total DL (Bytes)") AS total_download_data_bytes,
            SUM("Total UL (Bytes)") AS total_upload_data_bytes,
            SUM("Social Media DL (Bytes)" + "Social Media UL (Bytes)") AS total_social_media_data_bytes,
            SUM("Youtube DL (Bytes)" + "Youtube UL (Bytes)") AS total_youtube_data_bytes,
            SUM("Google DL (Bytes)" + "Google UL (Bytes)") AS total_google_data_bytes,
            SUM("Gaming DL (Bytes)" + "Gaming UL (Bytes)") AS total_gaming_data_bytes,
            SUM("Other DL (Bytes)" + "Other UL (Bytes)") AS total_other_data_bytes
        FROM
            xdr_data
        GROUP BY
            "IMSI"
        ORDER BY
            user_id;
        """

In [9]:
user_info = userInfoAgregation(query2)

In [10]:
agregate_df = user_info.agregation_by_user_info()

An error occurred: (psycopg2.errors.UndefinedColumn) column "Other DL" does not exist
LINE 12:             SUM("Other DL" + "Other UL") AS total_other_data...
                         ^

[SQL: 
        SELECT
            "IMSI" AS user_id,
            COUNT(*) AS num_xdr_sessions,
            SUM("Dur. (ms)") AS total_session_duration_ms,
            SUM("Total DL (Bytes)") AS total_download_data_bytes,
            SUM("Total UL (Bytes)") AS total_upload_data_bytes,
            SUM("Social Media DL (Bytes)" + "Social Media UL (Bytes)") AS total_social_media_data_bytes,
            SUM("Youtube DL (Bytes)" + "Youtube UL (Bytes)") AS total_youtube_data_bytes,
            SUM("Google DL (Bytes)" + "Google UL (Bytes)") AS total_google_data_bytes,
            SUM("Gaming DL (Bytes)" + "Gaming UL (Bytes)") AS total_gaming_data_bytes,
            SUM("Other DL" + "Other UL") AS total_other_data_bytes
        FROM
            xdr_data
        GROUP BY
            "IMSI"
        ORDER BY
      